### In this nootbook I will plot the performance of the 16-64 layer model (minimodel)

In [7]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from minimodel import data, metrics, model_builder, model_trainer

data_path = '../data'
weight_path = './checkpoints_16-64'
pretrained_weight_path = './checkpoints_16-320'
results_path = './results_16-64'

In [8]:
mouse_id = 5
file_name = "results_" + str(mouse_id)
results_file_path = os.path.join(results_path, file_name)
data = np.load(results_file_path)

FEV_scores_mini = data["FEV_scores"]
FEVE_scores_mini = data["FEVE_Scores"]
neurons_index_mini = data["neurons_index"]

FileNotFoundError: [Errno 2] No such file or directory: './results_16-64/results_5'

In [ ]:
print(f"FEV: {FEV_scores_mini}")
print(f"FEVE: {FEVE_scores_mini}")
print(f"index: {neurons_index_mini}")